First, let's add some imports that we'll need later.

We'll also generate some random arrays and write it to disk so we have some files to work with.

In [ ]:
import tempfile
from ray.data.datasource import RandomIntRowDatasource
import ray
import pandas as pd
import numpy as np

COLUMNS='ABCD'
SIZE_100MiB = 100 * 1024 * 1024
PARALLELISM = 8


def generate_example_files(size_bytes: int) -> str:
    tmpdir = tempfile.mkdtemp()
    ray.data.read_datasource(
        RandomIntRowDatasource(),
        n=size_bytes // 8 // len(COLUMNS),
        num_columns=len(COLUMNS)).write_parquet(tmpdir)
    return tmpdir

example_files_dir = generate_example_files(SIZE_100MiB)
# Try `ls` on this directory from shell to see what files were created.
print(example_files_dir)

# Ray Datasets

[Ray Datasets](https://docs.ray.io/en/master/data/dataset.html) are the standard way to load and exchange data in Ray libraries and applications. Datasets provide basic distributed data transformations such as map, filter, and repartition, and are compatible with a variety of file formats, datasources, and distributed frameworks.

![Dataset](https://docs.ray.io/en/master/_images/dataset.svg)

Ray Datasets implement [Distributed Arrow](https://arrow.apache.org/). A Dataset consists of a list of Ray `ObjectRef`s, which point to blocks. Each block holds a set of items in either an Arrow table or a Python list (for Arrow incompatible objects). Having multiple blocks in a dataset allows for parallel transformation and ingest of the data (e.g., into Ray Train for ML training).

![Arrow](https://docs.ray.io/en/master/_images/dataset-arch.svg)

Since a Ray Dataset is just a list of Ray object references, it can be freely passed between Ray tasks, actors, and libraries like any other object reference. This flexibility is a unique characteristic of Ray Datasets.

Compared to Spark RDDs and Dask Bags, Datasets offers a more basic set of features, and executes operations eagerly for simplicity. It is intended that users cast Datasets into more featureful dataframe types (e.g., ds.to_dask()) for advanced operations.

## Let's try it out

Let's try loading the data that we just wrote to disk with Ray Datasets. This will load the data in parallel for us, using Ray remote functions under the hood. The Dataset will be partitioned into blocks, with each block stored as an Arrow table.

In [ ]:
ds = ray.data.read_parquet(example_files_dir)
ds.show(limit=10)

Now that we have the data in Dataset form, we can start applying transformations! Let's start with `Dataset.map`. This operation creates a new Dataset by applying the given Python function to each record in the original Dataset.

**Task:** Fill out the lambda function in `Dataset.map` to create a new Dataset. Try getting the value in column `c_1` and dividing it by 10.

In [ ]:
# TODO: Fill out the map function to transform the dataset.
# Tip: You can use `row[column name]` to access the value in a given column.

ds.map(lambda row: "TODO: fill this in").show()

# Shuffling data with Ray Datasets

Let's try doing something a bit more useful.
A common operation that we need to do in machine learning is to randomly shuffle data between training epochs to improve overall accuracy.
This is a challenging task because it has to be pipelined with the training computation.
The problem gets even harder when the data doesn't fit in memory, and/or when the data is distributed across multiple nodes.

Shuffling data is challenging because it requires an all-to-all communication between processes or machines. Here's an illustration of how to do it using a MapReduce model:

![shuffle](https://miro.medium.com/max/453/1*nJYIs2ktVkqVsgSUCzfjaA.gif)

With Ray Dataset, we can easily shuffle the data and take advantage of multiple cores and/or nodes.
Again, under the hood, this gets implemented with Ray tasks, and the data gets shuffled in Ray's distributed memory store.
Let's try it here, although we'll only be using a single-node memory store.

**Task:** Evaluate the next cell. What is the difference between the two outputs?

In [ ]:
# What's the difference between these two outputs?
ds.show()
shuffled_ds = ds.random_shuffle()
shuffled_ds.show()